In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load dataset
df = pd.read_csv("/content/drive/My Drive/drug_no.csv")

In [ ]:
# Input widgets
condition_input = widgets.Text(
    placeholder="Enter condition (e.g., 'Depression', 'Anxiety'): ",
    description="🩺 Condition:",
    layout=widgets.Layout(width='50%')
)

search_box = widgets.Text(
    placeholder="Enter drug name or search keywords:",
    description="🔍 Query:",
    layout=widgets.Layout(width='50%')
)

show_all_checkbox = widgets.Checkbox(
    value=False,
    description='Show all matching reviews',
    indent=False
)

output = widgets.Output()

# Main search function
def search_reviews(condition, query, show_all):
    with output:
        output.clear_output()

        # Normalize condition
        condition = condition.strip().lower()
        if condition:
            filtered_df = df[df['condition'].str.lower().str.contains(condition, na=False)]
        else:
            filtered_df = df.copy()

        if not query:
            display(HTML("<b>Showing sample reviews (no query):</b><br><br>"))
            display(filtered_df.head(5)[['drugName', 'condition', 'review']])
            return

        # TF-IDF vectorization
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(filtered_df['review'])

        query_vector = vectorizer.transform([query])
        similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()

        filtered_df = filtered_df.copy()
        filtered_df['similarity'] = similarities

        # Sort by similarity
        filtered_df = filtered_df.sort_values(by='similarity', ascending=False)

        # Show all or top 5
        display_df = filtered_df if show_all else filtered_df.head(5)

        if not display_df.empty:
            display(HTML("<b>🔍 Matching Reviews:</b><br><br>"))
            for _, row in display_df.iterrows():
                highlighted_review = row['review']
                for keyword in query.lower().split():
                    highlighted_review = highlighted_review.replace(
                        keyword, f"<mark>{keyword}</mark>"
                    ).replace(
                        keyword.capitalize(), f"<mark>{keyword.capitalize()}</mark>"
                    )
                display(HTML(f"""
                    <div style="margin-bottom:1em; padding:0.5em; border:1px solid #ddd; border-radius:8px;">
                        <b>💊 Drug:</b> {row['drugName']}<br>
                        <b>🩺 Condition:</b> {row['condition']}<br>
                        <b>⭐ Similarity:</b> {row['similarity']:.2f}<br>
                        <b>📜 Review:</b><br>{highlighted_review}
                    </div>
                """))
        else:
            print("No relevant reviews found.")

# Wrapper to update output
def update_results(change=None):
    search_reviews(condition_input.value, search_box.value, show_all_checkbox.value)

# Observers (any change triggers update)
condition_input.observe(update_results, names='value')
search_box.observe(update_results, names='value')
show_all_checkbox.observe(update_results, names='value')

In [ ]:
# Display the UI
display(condition_input, search_box, show_all_checkbox, output)

Text(value='', description='🩺 Condition:', layout=Layout(width='50%'), placeholder="Enter condition (e.g., 'De…

Text(value='', description='🔍 Query:', layout=Layout(width='50%'), placeholder='Enter drug name or search keyw…

Checkbox(value=False, description='Show all matching reviews', indent=False)

Output()